# 1. Set Environment & Install Packages

## 1-1. Set Environment

In [1]:
import os
os.environ["HF_HOME"] = "/media/data/park"
os.environ["TRANSFORMERS_CACHE"] = "/media/data/park"
os.environ["HF_DATASETS_CACHE"] = "/media/data/park"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print(os.getcwd())

/home/s2hnpark/mentalhealth


In [2]:
from dotenv import load_dotenv
load_dotenv()

hf_token = os.getenv("HF_TOKEN")
print(f"HF_TOKEN: {hf_token[:10]}")

HF_TOKEN: hf_xZEqvBg


## 1-2. Install Packages

In [3]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    Trainer,
    pipeline,
    logging,
    DataCollatorForLanguageModeling
)

from peft import LoraConfig, get_peft_model
# from trl import SFTTrainer

/home/s2hnpark/.local/lib/python3.11/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


# 2. Reform Dataset according to Llama3 template

## In case of Llama3.2B, the following prompt template is used for the chat models

System Prompt (optional) to guide the model<br>
User prompt (required) to give the instruction<br>
Model Answer (required)<br>

\<s>[INST] \<\<SYS>> <br>
System Prompt <br>
\<\</SYS>> <br>

User Prompt [/INST] Model Answer \</s>"

In [4]:
import pandas as pd
df = pd.read_csv('train.csv')
print(df)

                                                Context  \
0     I'm going through some things with my feelings...   
1     I'm going through some things with my feelings...   
2     I'm going through some things with my feelings...   
3     I'm going through some things with my feelings...   
4     I'm going through some things with my feelings...   
...                                                 ...   
3507  My grandson's step-mother sends him to school ...   
3508  My boyfriend is in recovery from drug addictio...   
3509  The birth mother attempted suicide several tim...   
3510  I think adult life is making him depressed and...   
3511  I just took a job that requires me to travel f...   

                                               Response  
0     If everyone thinks you're worthless, then mayb...  
1     Hello, and thank you for your question and see...  
2     First thing I'd suggest is getting the sleep y...  
3     Therapy is essential for those that are feelin...  
4

In [5]:
df["new"] = df[df.columns[0:]].apply(
    lambda x: " [/INST] ".join(x.dropna().astype(str)),
    axis=1
)
print(f"{df['new'][0]}\n---------")
print(f"{df['new'][3511]}\n---------")
print(df)

I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone? [/INST] If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media.  Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is somehow terrible.Bad feelings a

In [6]:
df["new"] = df["new"].apply(lambda t: f"<s>[INST] {t} </s>")

print(df["new"].iloc[0])

<s>[INST] I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone? [/INST] If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media.  Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is somehow terrible.Bad 

Tokenize datasets with a model

In [7]:
model_id = "meta-llama/Llama-3.2-1B"

In [8]:
from datasets import Dataset, DatasetDict
data = {"new": df["new"][:]}
dataset = Dataset.from_dict(data)
print(dataset[0])
dataset

{'new': "<s>[INST] I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone? [/INST] If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media. \xa0Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is somehow

Dataset({
    features: ['new'],
    num_rows: 3512
})

In [9]:
dataset_dict = DatasetDict({"train": dataset})
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['new'],
        num_rows: 3512
    })
})


In [10]:
print(dataset_dict["train"][0])

{'new': "<s>[INST] I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone? [/INST] If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media. \xa0Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is somehow

In [11]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

In [12]:
# The numbers of tokens after Tokenizing.
def count_tokens(row: dict) -> int:
    return len(
        tokenizer(
            row["new"],
            add_special_tokens=True,
            return_attention_mask=False
        )["input_ids"]
    )

df["num_tokens"] = df.apply(count_tokens, axis=1)
df

,Context,Response,new,num_tokens
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb...",<s>[INST] I'm going through some things with m...,277
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see...",<s>[INST] I'm going through some things with m...,528
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...,<s>[INST] I'm going through some things with m...,150
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...,<s>[INST] I'm going through some things with m...,249
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...,<s>[INST] I'm going through some things with m...,150
...,...,...,...,...
3507,My grandson's step-mother sends him to school ...,Absolutely not! It is never in a child's best ...,<s>[INST] My grandson's step-mother sends him ...,209
3508,My boyfriend is in recovery from drug addictio...,I'm sorry you have tension between you and you...,<s>[INST] My boyfriend is in recovery from dru...,222
3509,The birth mother attempted suicide several tim...,"The true answer is, ""no one can really say wit...",<s>[INST] The birth mother attempted suicide s...,229
3510,I think adult life is making him depressed and...,How do you help yourself to believe you requir...,<s>[INST] I think adult life is making him dep...,145


In [13]:
# Tokenize all three datasets
def preprocess(dataset):
    tokenized = tokenizer(dataset["new"], truncation=True, padding="max_length", max_length=1024)
    return tokenized

In [14]:
tokenized_dataset = dataset_dict.map(preprocess, batched=True)
print(f"tokenized_dataset: {tokenized_dataset}")

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

tokenized_dataset: DatasetDict({
    train: Dataset({
        features: ['new', 'input_ids', 'attention_mask'],
        num_rows: 3512
    })
})


Quantizate model 


In [15]:
# Apply 4-bit quantization to reduce the model's memory footprint
# Qunatization is a technique to reduce the precision of the model's weigts, which can significantly reduce memory usage and speed up inference.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_type="float16",
    bnb_4bit_use_double_quant=True,
)

Unused kwargs: ['bnb_4bit_compute_type']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


In [16]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto", 
    offload_folder="offload",
)

In [17]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

qlora_model = get_peft_model(model, lora_config)
qlora_model.print_trainable_parameters()

trainable params: 1,703,936 || all params: 1,237,518,336 || trainable%: 0.1377


Configurate Training Arguments

In [18]:
training_args = TrainingArguments(
    output_dir="/media/data/park",
    logging_dir="/media/data/park",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=8,
    optim="adamw_bnb_8bit",
    save_total_limit=2,
    save_strategy="epoch",
    learning_rate=2e-4,
    fp16=True,
    num_train_epochs=1,
    ddp_find_unused_parameters=False,
    logging_steps=100,
    save_steps=500,
    report_to="wandb"
)

Train the Model

In [19]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [20]:
trainer = Trainer(
    model=qlora_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    data_collator=data_collator
)

In [21]:
trainer_stats = trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: lightgreenworld (lightgreenworld-universit-t-trier). Use `wandb login --relogin` to force relogin


/home/s2hnpark/.local/lib/python3.11/site-packages/bitsandbytes/nn/modules.py:430: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss


Save Fine-Tuned Model

In [22]:
# Test the fine-tuned model
qlora_model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Line

In [24]:
# Test prompt

generator = pipeline(
    "text-generation",
    model=qlora_model,
    tokenizer=tokenizer,
    return_full_text=True,
    # max_length=2048,
)

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'Jam

Use the text generation pipeline to ask questions like "What is a large language model?" <br>
Note that I'm formatting the input to match Llama3.2 prompt template

In [25]:

# Format the prompt according to Llama3.2 template
prompt = "<s>[INST] I barely sleep and I do nothing but think about how I'm worthless. [/INST]"

outputs = generator(
    prompt,
    max_new_tokens=512,
    do_sample=True,
    top_p=0.9,
    temperature=0.7,
)
print(outputs)

[{'generated_text': "<s>[INST] I barely sleep and I do nothing but think about how I'm worthless. [/INST] I think it is important to realize that you have some control over your thoughts. I also think it is important to realize that there is a difference between thoughts and feelings. Thoughts can be very powerful. It is very easy to think about how terrible you are. Thoughts can be very strong and intense. Thoughts can be very controlling. Thoughts can be very powerful.\xa0I also think that it is important to realize that you can have control over your thoughts. Thoughts can be very powerful, but they can also be very easy to control.\xa0If you can find a way to stop thinking about how worthless you are, and start thinking about what you have to be thankful for, and start thinking about what you are thankful for, and start thinking about what you are thankful for, and start thinking about what you are thankful for, and start thinking about how thankful you are, and start thinking abou

Save the new model in GGUF

In [27]:
from optimum.llama_cpp import LlamaCppModelForCausalLM

# 모델을 GGUF로 변환
LlamaCppModelForCausalLM.from_pretrained(
    "./fine-tuned/llama3.2-mentalhealth4",
    export=True,
    export_path="./gguf_model",
    quantization="q4_k_m"
)

ModuleNotFoundError: No module named 'optimum'